# Базово исследуем векторные авторегрессии  (ДЗ, Часть 1)

ВАШЕ ИМЯ, ФАМИЛИЯ

ВАША ГРУППА

Оценка (для проверяющего): Х из ??

В этой части 50 баллов.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
#all imports here, pls

## Разминка (15 баллов)

Для небольшой разминки немного посчитаем. Оператор $\text{Tr}$ -- это оператор следа. В заданиях ниже в силу алгебры ответ может быть записан не единственным образом.

### Задача 1 (1 балл)

Пусть $A,X,B \in \mathbb{R}^{n \times n}$. Выведите формулу для производной функции

$$
f(X) =\left(\text{Tr} ( AXB) \right)^5
$$

по матрице $X$. Выпишите результат в виде матрицы $D$, где $D_{ij}$  -- производная по $X_{ji}$.

ВАШЕ РЕШЕНИЕ И ОТВЕТ

### Задача 2 (2 балла)

Пусть $A,X,M \in \mathbb{R}^{n \times n}$. Выведите формулу для производной функции

$$
f(X) = \text{Tr} \left( A (X \odot M) \right)
$$

по матрице $X$. Выпишите результат в виде матрицы $D$, где $D_{ij}$  -- производная по $X_{ji}$.

Операция $\odot$ обозначает покомпонентное (Адамарово) произведение.

ВАШЕ РЕШЕНИЕ И ОТВЕТ

### Задача 3 (3 балла)

Пусть $A,X \in \mathbb{R}^{n \times n}, ~k \in \mathbb{Z}_{>0}$. Выведите формулу для производной функции

$$
f(X) = \text{Tr} \left( X^k  \right)
$$

по матрице $X$. Выпишите результат в виде матрицы $D$, где $D_{ij}$  -- производная по $X_{ji}$.

Используйте это, чтобы вывести формулу для производной

$$
f(X) = \text{Tr} \left( e^X\right),
$$

где матричная экспонента определяется через степенной ряд из матриц (ряд Тейлора). Выпишите результат в виде матрицы $D$, где $D_{ij}$  -- производная по $X_{ji}$.

ВАШЕ РЕШЕНИЕ и ОТВЕТЫ

### Задача 4 (3 балла)

Пусть $X \in \mathbb{R}^{n \times n}$. Выведите формулу для производной

$$
f(X) = \text{Tr} (cos(X)),
$$

где косинус матрицы определяется через степенной ряд из матриц (ряд Тейлора). Выпишите результат в виде матрицы $D$, где $D_{ij}$  -- производная по $X_{ji}$.


ВАШЕ РЕШЕНИЕ и ОТВЕТЫ

### Задача 5 (2 балла)

Дана сумма в соглашении Эйнштейна

$$
A_{zi}X_{ij}Z_{zk}X_{jk},
$$

где $X \in \mathbb{R}^{n \times n},~Z \in \mathbb{R}^{m \times n}, ~A \in \mathbb{R}^{m \times n}$.

Запишите, чему она равна, используя матричные операции (матричное и покомпонентное произведение, след, степени, транспонирование).

При формулировке ответа укажите какие размерности имеет результат и в каком порядке вы их выписали (первая размерность -- количество строк, вторая -- столбцов).

ВАШЕ РЕШЕНИЕ И ОТВЕТЫ

### Задача 6 (2 балла)

Дано выражение из двух сумм в соглашении Эйнштейна (в каждом слагаемом свой набор свободных и суммируемых индексов)

$$
A_{ij}A_{kj}B_{kt} +  A_{ki}B_{tk}
$$

где $A \in \mathbb{R}^{m \times n},~ B \in \mathbb{R}^{m \times m}$.

Запишите, чему она равна, используя матричные операции (матричное и покомпонентное произведение, след, степени, транспонирование).

При формулировке ответа укажите какие размерности имеет результат и в каком порядке вы их выписали (первая размерность -- количество строк, вторая -- столбцов).

ВАШЕ РЕШЕНИЕ И ОТВЕТЫ

### Задача 7 (2 балла)

Запишите в виде суммы Эйнштейна выражение

$$
\text{Tr}\left( A \odot B + (BA) \odot C\right).
$$

Операция $\odot$ обозначает покомпонентное (Адамарово) произведение.

ВАШЕ РЕШЕНИЕ И ОТВЕТЫ

## Модель векторной авторегрессии с экзогенными факторами (15 баллов)

Под экзогенными факторами мы будем понимать некоторый сигнал $u_t$, который мы не знаем заранее, но всегда знаем в момент $t$. В частности, это может быть управление - регистрируемое воздействие на систему (поворот руля, уввеличение или уменьшение тяги, движения джойстика и так далее).

Естественно, если такой сигнал есть, хочется его учесть про оценке параметров модели.

Обычная модель VAR(1) дополняется ещё одним слагаемым:

$$
X_{t+1} = AX_t + Bu_t + \varepsilon_{t+1}, \quad \varepsilon_t \sim N(0,\Sigma).
$$

Матрица $B$ отражает влияние управления на динамику, это связано с физической моделью. Как правило, $B$ известна, но её можно и оценивать.

В этом задании мы рассмотрим движение идеального робота (упрощённая кинематика), который двигается на плоскости $(x,y)$ и управляется внешним образом с джойстика.

Уравнения для координат и скоростей выглядят следующим образом:

$$
X_{t+1} = X_t + c_x V^x_t \Delta t + \varepsilon^x_{t+1},
$$

$$
Y_{t+1} = Y_t + c_y V^y_t \Delta t + \varepsilon^y_{t+1},
$$

$$
V^x_{t+1} = V^x_{t} + u^x_t \Delta t + \varepsilon^{u,x}_{t+1}
$$

$$
V^y_{t+1} = V^y_{t} + u^y_t \Delta t + \varepsilon^{u,y}_{t+1},
$$

где $\varepsilon$ обозначают независимые шумы, каждый с нулевым матожиданием и со своей дисперисей, не зависящей от времени, а $c_x,c_y$ некоторые неизвестные константы.


### Уравнение в векторной форме

Запишите динамику наблюдений в виде VAR(1) с экзогенными факторами (state-space форма):

$$
X_{t+1} = A X_t + Bu_t + \varepsilon_{t+1}, \quad \varepsilon_t \sim N(0, \Sigma).
$$

Определите, что такое состояние $X_t$, управление $u_t$, а также выпишите явно матрицы $A,B$ и $\Sigma$.

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ

### Оцениваем модель

Запишите лог-правдоподобие для модели VAR(1) с экзогенными факторами. В чём отличие от обычной модели VAR(1)?

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ

Теперь выведите формулы для оценок $A,\Sigma$, предполагая $u_t$ известными в момент $t$.

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ

...

ОТВЕТЫ: ....

### Пробуем на данных

Наша команда попробовала погонять такого робота и собрала набор данных, включающих в себя в том числе управления.

Оцените матрицу динамики $A$ и ковариацию шума $\Sigma$ по этим данным.

Для этого понадобится ещё один VAR.

In [ ]:
class VAREx:

    def __init__(self, A=None, B=None, Sigma=None, d=2):
        """VAREx(1) Model
        X_t = A X_{t-1} + B u_{t-1} + Eps_t,  Eps is normal WN

        Args:
            A (float[]) optional): A matrix (d,d). Defaults to None.
            B (float[]) optional): B matrix (d,?). Defaults to None.
            Sigma (float[], optional): Noise  covariance (d,d). Defaults to None.
            d (int, optional): VAR dimensionality. Defaults to 2.

        Raises:
            Exception: If some inconsistency found
        """        
        #some dimension inconsistencies may happen, beware
        #self.Sigma =
        #self.d = 
        #??raise Exception("inconsistent init data")
        #self.A = 
        #self.B = 
        

    def __str__(self):
        msg = f"Model VAR(1), dim={self.d} \n"
        msg = msg+ f"A= {self.A},\n"
        msg = msg + f"Sigma= {self.Sigma},\n"
        msg = msg + f"B= {self.B},\n"
        return  msg
    
    def simulate(self, startValue, controls, NSteps=15, NTraj = 10):
        """Simulates Trajectories given startValues

        Args:
            startValue (float[]): init value, of shape (d,)   
            NSteps (int, optional): time steps to compute. Defaults to 15.
            NTrajs (int, optional): number of trajectories to compute. Defaults to 15.

        Returns:
            vals float[] (NTraj,d,Nsteps+1)  -- simulated values
        """        
        #vals =  ???
        return vals
    

    def fit(self, xs, controls):
        """Fits the model to the data

        Args:
            xs (float[]): observed data (d,T)
        """        

        self.A = ???# (d,pd)
        self.Sigma = ???# (d,d)

Быстрый тест, чтобы убедится, что всё работает.

In [ ]:
A = np.array([[1,0.2],[0,1]])
B = np.array([[0,2],[2,0]])
Sigma = np.array([[1,0.2],[0.2,0.3]])*0.1
modelEx = VAREx(A,B,Sigma,d=2)
NSteps=100
controls = np.concatenate([np.arange(NSteps)[None,:]**2,np.arange(NSteps)[None,:]**2],axis=0)*1e-5

simData = modelEx.simulate(startValue=np.array([0,0]), 
                           controls = controls,
                           NSteps=NSteps, NTraj=1).squeeze()

In [ ]:
#plot
f, axs = plt.subplots(simData.shape[0],1, figsize=(15,7*simData.shape[0]))

for dd in range(simData.shape[0]):
    axs[dd].grid()
    axs[dd].set_title(f"Состояния X (id={dd})", fontsize=16)
    axs[dd].set_xlabel("T", fontsize = 15)
    axs[dd].plot(np.arange(simData.shape[-1]), simData[dd,:])

fg2, ax2 = plt.subplots(figsize=(8,7))

ax2.grid()
ax2.set_title("Управления", fontsize=16)
ax2.plot(controls[0,:])
ax2.plot(controls[1,:])
ax2.legend(["id0","id1"])
plt.show()

In [ ]:
modelEx2 = VAREx(d=2, B=modelEx.B)
modelEx2.fit(simData,controls)

In [ ]:
print(modelEx)
print("******")
print(modelEx2)
print(f"AErr: {np.sum(np.abs(modelEx.A - modelEx2.A))}")
print(f"SigmaErr: {np.sum(np.abs(modelEx.Sigma - modelEx2.Sigma))}")

Погрешность порядка 1e-2..1e-2 -- это ОК, значит вы вроде правильно сделали.

Загрузите теперь тестовый набор данных ``exoData.pickle`` (это pickle, посмотрите какие там есть ключи).

In [ ]:
## LOAD IT

Ниже по этим данным нарисуйте несколько графиков.

1. Траекторию робота на плоскости $X,Y$.
2. Четыре графика для всех состояний.
3. Отдельный график для двух управлений.

In [ ]:
##ВАШИ ГРАФИКИ ТУТ

Попробуйте по данным оценить модель $VAR(1)$ с экзогенными факторами, используя ваш код выше и выведите полученную оценку.

In [ ]:
# ВАШ FIT

ВАШИ КОММЕНТАРИИ И ГРАФИКИ ТУТ

## VAR(p)  (20 баллов)

До сих пор перед нами была модель VAR(1), векторная авторегрессия первого порядка. Теперь посмотрим, что изменится в технике, когда мы добавим более старые наблюдения:

$$
X_{t} = A_1 X_{t-1} + ... + A_p X_{t-p} +  \varepsilon_t, \quad \varepsilon_t \sim N(0,\Sigma).
$$

Как и раньше, мы предполагаем, что нам дано $X_1,..,X_n$ -- наблюдения из модели ряда и наша задача -- оценить матрицы $A_1,..,A_p$ и матрицу $\Sigma$. Будем двигаться по шагам.

Можно обратить внимание, что по сути модель очень похожая на всё остальное. Если мы обозначим

$$
\widetilde{A} = \left[A_1 \vert .. \vert A_p \right], \quad \widetilde{X}_{t-} = \left[ X_{t-1}^T, ~..,~ X_{t-p}\right]^T, 
$$

то уравнение примет почти привычный вид

$$
X_{t} = \widetilde{A} \widetilde{X}_{t-} + \varepsilon_t.
$$

Дальше дело техники -- оценить $\widetilde{A}$ и $\Sigma$.

### Правдоподобие


Выпишите функцию лог-правдоподобия. В этой модели мы предполагаем $X_1,..,X_p$ значения фиксированными константами, так как в модели используется более старые наблюдения по сравнению с $VAR(1)$. В чём отличие от случая VAR(1)?



ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ

### Оцениваем $\Sigma$

Как ни странно, оценка для $\Sigma$ обычно получается в очень лаконичном виде при известном $\widetilde{A}$

Выведите оценку ММП для $\Sigma$.

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ

### Оцениваем $\widetilde{A}$

Теперь выведите оценку ММП для матрицы $\widetilde{A}$, из которой уже легко достаются отдельные матрицы $A_j$.


ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ

### Проверяем на данных

Имплементация класса уже должна быть привычной, но потребуется некоторые детали обобщить. Обратите особенное внимание на симуляцию, чтобы не перепутать порядок $A_j$, а также будьте внимательны при записи формул для оценок.

Есть два пути: что-то сделать средствами циклов питона либо всё сделать лаконичнее с помощью ``einsum``  и  ``reshape``ов (очень рекомендуем второе).

In [ ]:
class VARp:

    def __init__(self, listA=None, sigma=None, p=1, d=2):
        """VAR Model
        X_t = A_1 X_{t-1} + .. + A_p X_{t-p} + Eps_t,  Eps is normal WN

        Args:
            listA (list(float[]), optional): list of A_j matrices (d,d). Defaults to None.
            sigma (float[], optional): Noise  covariance (d,d). Defaults to None.
            p (int, optional): VAR order. Defaults to 1.
            d (int, optional): VAR dimensionality. Defaults to 2.

        Raises:
            Exception: If some inconsistency found
        """        
        #??
        # beware of dimension inconsistencies
        # YOUR CODE

    def __str__(self):
        msg = f"Model VAR({self.p}), dim={self.d} \n"
        #msg = msg + whatever text you want reflecting the parameters
        return  msg
    
    def simulate(self, startValues, NSteps=15, NTraj = 10):
        """Simulates Trajectories given startValues

        Args:
            startValues (list(float[])): init values, of shape (d,) each , ordered t-p to t-1 
            NSteps (int, optional): time steps to compute. Defaults to 15.
            NTrajs (int, optional): number of trajectories to compute. Defaults to 15.

        Returns:
            vals float[] (NTraj,d,Nsteps+self.p)  -- simulated values
        """        
        #init processing
        vals =?? 
        return vals
    

    def fit(self, xs):
        """Fits the model to the data

        Args:
            xs (float[]): observed data (d,T)
        """        

        ##????? YOUR CODE

Быстрый тест на работоспособность:

In [ ]:
#listA assumed from t-1 till t-p order
listA = [np.array([[0.5,0],[-0.1,0]]),np.eye(2)*1e-1,np.array([[0.1,0],[-0.01,0]])]# np.array([[0.05,0.05],[0.2,0.1]]), np.array([[0,0.01],[-0.01,0]]) ]
sigma = np.random.standard_normal(size=(2,2))
sigma = sigma@(sigma.T) * 1e-3
varpTest = VARp(listA, sigma, p=3,d=2)

print(varpTest)

In [ ]:
startValues = [np.array([-1,1]),np.array([0,0]),np.array([1,-1])]
simData = varpTest.simulate(startValues,NSteps=1500,NTraj=5)

In [ ]:
f, axs = plt.subplots(simData.shape[1],1, figsize=(15,5*simData.shape[1]))

for dd in range(simData.shape[1]):
    axs[dd].grid()
    axs[dd].set_title(f"Состояния X (id={dd})", fontsize=16)
    axs[dd].set_xlabel("T", fontsize = 15)
    axs[dd].plot(np.arange(simData.shape[-1]), simData[:,dd,:].T)

In [ ]:
varpTest2 = VARp(p=3,d=2)
varpTest2.fit(simData[0,...])
print(varpTest)
print(varpTest2)

Если попробуете поэкспериментировать, то поймёте, что оценивать точно такую модель крайне сложно из-за большого количества зависимостей между координатами и временами. Оценка параметров -- это ещё не всё и статистические критерии (тоже нетриввиальные) позволяют определить, где действительно может быть 0.

Если тут у вас ошибка в первом ненулевом знаке и дальше, то вроде ок.

С задачей оценки хорошо справляется фильтр Калмана, если его правильно применить, но формально это уже Байесовский подход.

Загрузите набор данных ``vardata.csv``. и напечатайте график.

In [ ]:
import pickle
with open("./vardata.pickle","rb") as f:
    dd=pickle.load(f)
vals = dd["values"]

In [ ]:
#import pickle
#with open("./vardata.pickle","wb") as f:
#    pickle.dump({"values": vals.squeeze(), "desc": "Набор данных из VAR(2). Первая размерность -- это время, нулевая -- векторная размерность (размерность X)."},f)

In [ ]:
#plot
f, axs = plt.subplots(vals.shape[0],1, figsize=(15,7*vals.shape[0]))

for dd in range(vals.shape[0]):
    axs[dd].grid()
    axs[dd].set_title(f"AR Симуляции (id={dd})", fontsize=16)
    axs[dd].set_xlabel("T", fontsize = 15)
    axs[dd].plot(np.arange(vals.shape[-1]), vals[dd,:])

plt.show()

Видно, что это не набор независимых наблюдений.

Попробуйте оценить модель VAR(2) по данному набору наблюдений и напечатайте параметры.

In [ ]:
modelFit = VARp(p=2,d=2)
modelFit.fit(??)

print("My Model estimation:")
print(modelFit)
#print(model)
#print(np.sum(np.abs(modelFit.Atilde - model.Atilde)))
d=2
valsEst = modelFit.simulate([np.ones([d]), -np.ones([d])],NSteps=500, NTraj=5)

In [ ]:
f, (axs1, axs2) = plt.subplots(vals.shape[0],2, figsize=(30,7*vals.shape[0]))

for dd in range(vals.shape[0]):
    axs1[dd].grid()
    axs1[dd].set_title(f"AR Симуляции (id={dd})", fontsize=16)
    axs1[dd].set_xlabel("T", fontsize = 15)
    axs1[dd].plot(np.arange(vals.shape[-1]), vals[dd,:])

for dd in range(valsEst.shape[1]):
    axs2[dd].grid()
    axs2[dd].set_title(f"AR Симуляции Оценённой модели (id={dd})", fontsize=16)
    axs2[dd].set_xlabel("T", fontsize = 15)
    axs2[dd].plot(np.arange(valsEst.shape[-1]), valsEst[:,dd,:].T,"--", linewidth=0.5)
    axs2[dd].plot(np.arange(vals.shape[-1]), vals[dd,:], linewidth=1.3)